In [1]:
import json # import json module
import os
import os.path as pth
from glob import glob
import cv2
import matplotlib.image as img
import numpy as np
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline 
from PIL import Image 

import matplotlib.image as mpimg
import gc
import seaborn as sns
import sys

print('ready')


ready


In [2]:
train_df = pd.read_csv('./data/kaggle_FGVC/train.csv')


display(train_df)

,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
0,00000663ed1ff0c4e0132b9b9ac53f6e,6068157 7 6073371 20 6078584 34 6083797 48 608...,5214,3676,6,"115,136,143,154,230,295,316,317"
1,00000663ed1ff0c4e0132b9b9ac53f6e,6323163 11 6328356 32 6333549 53 6338742 75 63...,5214,3676,0,"115,136,142,146,225,295,316,317"
2,00000663ed1ff0c4e0132b9b9ac53f6e,8521389 10 8526585 30 8531789 42 8537002 46 85...,5214,3676,28,163
3,00000663ed1ff0c4e0132b9b9ac53f6e,12903854 2 12909064 7 12914275 10 12919485 15 ...,5214,3676,31,"160,204"
4,00000663ed1ff0c4e0132b9b9ac53f6e,10837337 5 10842542 14 10847746 24 10852951 33...,5214,3676,32,219
...,...,...,...,...,...,...
333396,fffe20b555b98c3c1f26c8dfff275cbc,2712731 8 2715725 23 2718719 39 2721713 55 272...,3000,2001,28,163
333397,ffffbf7014a9e408bfbb81a75bc70638,71179 1 71678 3 72178 4 72678 4 73178 5 73679 ...,500,375,33,NaN
333398,ffffbf7014a9e408bfbb81a75bc70638,116648 5 117148 16 117648 22 118148 26 118647 ...,500,375,31,157
333399,ffffbf7014a9e408bfbb81a75bc70638,67711 1 68210 1 68709 2 69204 2 69208 3 69705 ...,500,375,31,157


In [3]:
with open('./data/kaggle_FGVC/label_descriptions.json') as json_file:
    label_data = json.load(json_file)



print(label_data['categories'])
print(' ')
print(label_data['attributes'])


[{'id': 0, 'name': 'shirt, blouse', 'supercategory': 'upperbody', 'level': 2}, {'id': 1, 'name': 'top, t-shirt, sweatshirt', 'supercategory': 'upperbody', 'level': 2}, {'id': 2, 'name': 'sweater', 'supercategory': 'upperbody', 'level': 2}, {'id': 3, 'name': 'cardigan', 'supercategory': 'upperbody', 'level': 2}, {'id': 4, 'name': 'jacket', 'supercategory': 'upperbody', 'level': 2}, {'id': 5, 'name': 'vest', 'supercategory': 'upperbody', 'level': 2}, {'id': 6, 'name': 'pants', 'supercategory': 'lowerbody', 'level': 2}, {'id': 7, 'name': 'shorts', 'supercategory': 'lowerbody', 'level': 2}, {'id': 8, 'name': 'skirt', 'supercategory': 'lowerbody', 'level': 2}, {'id': 9, 'name': 'coat', 'supercategory': 'wholebody', 'level': 2}, {'id': 10, 'name': 'dress', 'supercategory': 'wholebody', 'level': 2}, {'id': 11, 'name': 'jumpsuit', 'supercategory': 'wholebody', 'level': 2}, {'id': 12, 'name': 'cape', 'supercategory': 'wholebody', 'level': 2}, {'id': 13, 'name': 'glasses', 'supercategory': 'head

In [4]:
print(len(train_df['EncodedPixels'][0]))

print(train_df['EncodedPixels'][0])


21303
6068157 7 6073371 20 6078584 34 6083797 48 6089010 62 6094223 72 6099436 76 6104649 80 6109862 85 6115075 89 6120288 93 6125501 98 6130714 102 6135927 106 6141140 111 6146354 114 6151567 118 6156780 123 6161993 127 6167206 131 6172419 136 6177632 140 6182845 144 6188058 149 6193271 153 6198484 157 6203697 162 6208910 166 6214124 169 6219337 174 6224550 178 6229763 182 6234976 187 6240189 191 6245402 195 6250615 200 6255828 204 6261041 208 6266254 213 6271467 218 6276680 224 6281893 229 6287107 233 6292320 238 6297533 244 6302746 249 6307959 254 6313172 259 6318385 265 6323598 270 6328811 275 6334024 280 6339237 286 6344450 291 6349663 296 6354877 300 6360090 306 6365303 311 6370516 316 6375729 322 6380942 327 6386155 332 6391368 337 6396581 343 6401794 348 6407007 353 6412220 358 6417433 364 6422647 368 6427860 373 6433073 378 6438286 384 6443499 389 6448712 394 6453925 399 6459138 405 6464351 410 6469564 415 6474777 420 6479990 426 6485203 431 6490416 436 6495630 441 6500843 446

In [5]:
display(pd.DataFrame([train_df['Height'].describe(), train_df['Width'].describe()]).T.loc[['max', 'min', 'mean']])

num_classes = len(label_data['categories'])
num_attributes = len(label_data['attributes'])
print(f'Total # of classes: {num_classes}')
print(f'Total # of attributes: {num_attributes}')

,Height,Width
max,8688.000000,10717.00000
min,296.000000,151.00000
mean,2235.524021,1761.97925


Total # of classes: 46
Total # of attributes: 294


In [6]:
categories_df = pd.DataFrame(label_data['categories'])
attributes_df = pd.DataFrame(label_data['attributes'])
display(categories_df)

,id,name,supercategory,level
0,0,"shirt, blouse",upperbody,2
1,1,"top, t-shirt, sweatshirt",upperbody,2
2,2,sweater,upperbody,2
3,3,cardigan,upperbody,2
4,4,jacket,upperbody,2
5,5,vest,upperbody,2
6,6,pants,lowerbody,2
7,7,shorts,lowerbody,2
8,8,skirt,lowerbody,2
9,9,coat,wholebody,2


In [7]:
my_category_list = [0,1,2,3,4,5,6,7,8,9,10,11,14,21,23]

for i in my_category_list:
    display(train_df[train_df['ClassId'] == i])
    print(' ')
    
tshirt_df = train_df[train_df['ClassId'] == 1]

,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
1,00000663ed1ff0c4e0132b9b9ac53f6e,6323163 11 6328356 32 6333549 53 6338742 75 63...,5214,3676,0,"115,136,142,146,225,295,316,317"
119,0010ef2da53cd76ee26122a65f6e76ee,5153671 6 5158276 20 5162882 31 5167488 42 517...,4608,3072,0,"115,135,142,146,225,295,316,317"
222,002bc82e34205ecf66052749c8ff9e32,1303221 6 1305550 18 1307880 29 1310209 41 131...,2334,1522,0,"115,136,145,146,225,295,316,317"
239,0036f906cb15e240f113184b5eed8c69,116510 2 117157 6 117803 11 118445 20 119089 2...,650,433,0,"115,136,145,147,225,295,316,317"
283,0039366c35ed742e5bfc6a278b699720,752506 23 754090 70 755675 100 757265 110 7588...,1600,1067,0,"115,225,295,316,317"
...,...,...,...,...,...,...
333189,ffdc587a4b41cc4ff12c56e4fe06086b,249513 2 250713 6 251914 9 251929 5 253114 21 ...,1200,801,0,"147,316"
333211,ffe011e941a22c02ef0be02a922d44b7,6921499 3 6926776 9 6932054 15 6937331 19 6942...,5278,3519,0,"115,136,142,146,225,295,316,317"
333291,fff11e31ec0146bd469e5e3afa14a37a,289982 6 291001 16 292021 21 293043 25 294066 ...,1024,633,0,"115,136,295,316"
333309,fff132f7ce55db363a1afc4fda330ffc,2653567 3 2656560 11 2659556 15 2662553 18 266...,3000,1997,0,"115,136,142,146,225,295,316,317"


,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
10,0000fe7c9191fba733c8a69cfaf962b7,1343707 9 1346138 27 1348569 44 1351000 62 135...,2448,2448,1,"0,115,145,146,295,316,317"
24,00048c3a2fb9c29340473c4cfc06424a,257702 10 258670 26 259640 36 260572 6 260610 ...,975,650,1,"11,115,135,145,146,295,316,317"
31,0006ea84499fd9a06fefbdf47a5eb4c0,73327 6 74328 18 75330 37 76335 63 77340 88 78...,1000,667,1,"11,135,146,295"
37,000775b6b5e27b011dc8bba2d7b85211,309670 11 311259 33 312848 55 314440 72 316033...,1600,1067,1,"115,136,145,295,314,317"
54,000b3ec2c6eaffb491a5abb72c2e3e26,458445 17 460731 51 463017 85 465303 115 46759...,2310,1536,1,"0,115,136,145,147,295,314,317"
...,...,...,...,...,...,...
333273,ffec8295f37df6ea12eecbb60d2c23d4,2599516 1 2602970 3 2606425 4 2609879 6 261333...,3456,2304,1,"0,115,136,145,146,295,316,317"
333322,fff36676cdac2dd8d057783f0e9f24cc,1401684 4 1403933 12 1403962 4 1406137 4 14061...,2256,1500,1,"0,317"
333336,fff3ae4585d776922829ceb887789be4,122780 3 123500 6 123511 7 124220 22 124940 24...,720,480,1,"14,115,136,145,147,295,316,317"
333351,fff65379ba62e3f6498a20e9cc2a99fe,5913102 5 5917131 10 5921163 12 5925195 14 592...,4032,3024,1,"0,115,136,146,295,301,317"


,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
296,003a6ca3eaa7886da61ed2de2c5e8d27,5158311 19 5161042 23 5163772 28 5163869 5 516...,2733,3630,2,"136,147,295,316,317"
298,003ad8a37d2190bd944a8968fb0906e2,18556042 9 18561335 28 18566629 46 18571922 65...,5304,7952,2,"115,136,142,147,295,316,317"
314,003c831a9e84ee589d67c4d91786f525,48287 3 48882 9 49468 24 50049 44 50635 64 512...,600,371,2,"115,135,142,147,295,316,317"
580,00689c9d05028aad72aa2a9dfccbd491,8143995 5 8147985 15 8151974 26 8155964 36 815...,4000,6000,2,"115,136,295,316,317"
664,007e66e7c2864eb3c1ef95cd3ab52687,5093384 3 5098565 11 5103747 17 5108928 25 511...,5184,3456,2,"115,136,145,146,295,316,317,328"
...,...,...,...,...,...,...
332358,ff3c229ab653661ecd5c857ef20e77db,92626 4 93271 11 93917 14 94564 16 95211 18 95...,650,433,2,"115,136,142,149,295,316,317"
332663,ff73f17eae6e03385dac85aa2e0ad415,3324489 3 3327332 10 3330176 16 3333021 21 333...,2848,4288,2,"115,136,145,146,316"
332943,ffa9ee64af28f111006fbdabf6db09e2,3117 63 4848 153 6616 206 8384 259 10012 69 10...,1800,1183,2,"115,145,147,295,316,317,328"
333213,ffe011e941a22c02ef0be02a922d44b7,3285868 49 3291072 147 3296311 210 3301584 238...,5278,3519,2,"115,136,145,146,295,316,317"


,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
44,000aac3870ea7c59ca0333ffa5327323,8621972 2 8625970 5 8629968 9 8633966 13 86379...,4000,6000,3,"115,136,145,147,225,295,316,317"
478,00573a530f2e9e4c203681547e259f52,13173 8 13958 25 14744 41 15529 52 16314 57 17...,789,800,3,"115,136,145,146,295,316,317"
486,0058f13eef5b1a66e42666bbbb834ca9,81846 13 81929 39 82831 37 82893 76 83816 153 ...,1000,800,3,"115,145,146,225,281,312,319"
679,007fb0d71cfa24fe496363d5198f706d,36357 3 37004 7 37374 9 37650 13 38015 22 3823...,650,433,3,"115,137,142,149,295,316,317"
1192,00f43ac0d027955e5fe6c99bd412fd8f,3248208 5 3251198 16 3254187 27 3257177 37 326...,3000,2000,3,"115,136,145,149,225,295,316,317"
...,...,...,...,...,...,...
331727,feb527d59d79e3b191ea21a06a3a72b3,82532 1 83131 3 83729 5 84328 7 84926 10 85525...,600,450,3,"115,136,145,151,225,295,316,317"
332161,ff0c12c7e53961891d3930a0a61c698e,4203322 1 4206584 5 4209846 8 4213108 11 42163...,3264,4912,3,"147,295,316,317"
333148,ffd7077c30443a0d3f1cb1e0588bf41c,6492408 2 6497591 8 6502774 13 6507958 18 6513...,5184,3456,3,"115,149,295,316,328"
333269,ffebc44ef1bbb5067632bfa26383e425,1873710 1 1876708 3 1879707 4 1882705 6 188570...,3000,2000,3,"115,136,145,147,295,316,317"


,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
8,00000663ed1ff0c4e0132b9b9ac53f6e,4566382 8 4571592 25 4576803 41 4582013 58 458...,5214,3676,4,"17,115,136,145,149,225,295,311,317"
62,000c5bb8ca785bce8d00e3a6c7802a8c,322102 1 323125 2 324148 3 325171 4 326194 5 3...,1024,683,4,"115,135,142,146,308,317"
92,000e973c99dc090afd7898c93daf0dbc,4796962 5 4801882 14 4806801 24 4811721 33 481...,4928,3264,4,"17,115,136,142,146,295,316,328"
101,000e973c99dc090afd7898c93daf0dbc,2022977 2 2027903 8 2032828 14 2037754 19 2042...,4928,3264,4,"28,136,295,317"
167,001d7807a696231045860eaee97e825b,247017 4 248399 11 249780 16 251162 18 252543 ...,1383,900,4,"115,116,142,147,229,283,311,317"
...,...,...,...,...,...,...
333242,ffe8150a195e3aaa72640d7730a811a1,7279531 8 7283162 24 7286797 36 7290436 44 729...,3648,5472,4,"17,136,142,146,225,295,308,328"
333302,fff11e31ec0146bd469e5e3afa14a37a,195055 19 196068 56 197080 81 198092 94 199105...,1024,633,4,"17,115,136,142,148,225,295,311,317"
333319,fff132f7ce55db363a1afc4fda330ffc,2005308 93 2008300 194 2011293 206 2014288 217...,3000,1997,4,"17,115,136,142,147,225,295,311,322"
333334,fff3ae4585d776922829ceb887789be4,98199 15 98221 27 98905 69 99620 79 100334 90 ...,720,480,4,"21,115,136,142,146,225,295,298,317"


...

,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
12,0002ec21ddb8477e98b2cbb87ea2e269,2287509 4 2290504 12 2290588 26 2293501 18 229...,3000,1997,10,"102,128,142,150,229,295,301,318"
15,0002f5a0ebc162ecfb73e2c91e3b8f62,435 132 1002 132 1569 132 2136 132 2703 132 32...,567,400,10,"108,115,119,141,155,229,286,316,317"
18,0004467156e47b0eb6de4aa6479cbd15,132663 8 133396 25 134130 41 134868 53 135611 ...,750,500,10,"115,141,155,295,305,317"
42,000aac3870ea7c59ca0333ffa5327323,8971124 17 8975101 49 8979077 82 8983053 116 8...,4000,6000,10,"104,115,129,145,149,295,308,325"
48,000b3a87508b0fa185fbd53ecbe2e4c6,291578 3 292856 8 294133 13 295410 19 296687 2...,1280,852,10,"112,114,128,142,151,295,311,317,322"
...,...,...,...,...,...,...
333261,ffebc44ef1bbb5067632bfa26383e425,2762370 9 2765353 26 2768335 44 2771318 62 277...,3000,2000,10,"113,115,117,141,153,229,295,316,317"
333281,ffed4d2a46f54337b99357dac5783564,293022 7 294518 16 296015 19 297511 24 299007 ...,1500,849,10,"114,129,145,150,295,296,317"
333358,fff8d9bd5ad4354b66355dd797942b42,408993 3 410489 8 411985 13 413481 19 414977 2...,1500,1000,10,"101,115,129,142,152,229,295,309,340"
333375,fffc631acce2e28e1628de685d40c980,210058 15 211030 18 212002 21 212974 24 213946...,975,650,10,"103,145,150,225,295,298,317"


,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
1386,011ea8ad74020b797bb70dff32eb965d,1454723 7 1457712 22 1460700 37 1463689 51 146...,3000,2000,11,"115,125,141,155,295,301,325"
2846,02389d8de4ec5f040c639282732b53a1,162399 1 163478 3 164556 6 164589 8 165634 9 1...,1080,1080,11,"115,126,142,154,229,295,305,306,317"
3175,026a9d4e51588f0973eea33bb81c512a,1542006 4 1543999 12 1545991 20 1547983 28 154...,1995,3000,11,"115,295,317"
3776,02e0e61e9828870674129227de56e45c,1231735 9 1234134 26 1236533 44 1238932 61 124...,2408,1673,11,"115,125,141,154,295,316,317"
3854,02f6c36c2a99353edadbeb93931346f8,1294686 38 1297639 113 1300612 169 1303513 16 ...,3000,2000,11,"115,135,141,155,234,295,316,317"
...,...,...,...,...,...,...
330948,fe05e6d28b36fee9b3b9227d613e80b5,275926 9 277524 13 279120 19 280713 29 282306 ...,1600,1067,11,"115,125,137,141,155,229,295,298,317"
331168,fe31fb1a96780626756d473ba5a87af4,7803789 63 7809121 71 7814453 78 7819785 86 78...,5336,3837,11,"115,136,142,149,229,295,316,331"
331561,fe8702c3e0b29091a557a00eb6056570,2250178 7 2253666 21 2257154 35 2260642 49 226...,3500,2333,11,"115,125,142,155,229,295,316,317"
332808,ff89d127a396bc4e269161fcb0debafc,157993 7 158979 21 159966 34 160952 48 161938 ...,1000,667,11,"115,128,136,141,295,305,325"


,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
103,000e973c99dc090afd7898c93daf0dbc,5889525 12 5894431 36 5899346 51 5904270 57 59...,4928,3264,14,NaN
710,0084d5a59b959293507052f5679d1be5,6663362 49 6668544 54 6673726 59 6678908 65 66...,5184,3456,14,NaN
791,0091ae902cb738e4ca06038c2654fa00,714885 7 716680 20 718474 33 720269 46 722065 ...,1800,1200,14,NaN
1018,00cd5bb501e031aeeeeb337fb3f5b6d8,167360 2 168159 5 168959 5 169758 6 170557 7 1...,800,534,14,NaN
1088,00df4e94fea28af045f51846858a2e3e,7960563 6 7966169 18 7971775 31 7977381 43 798...,5616,3744,14,NaN
...,...,...,...,...,...,...
332930,ffa52a33c743c48512a85169251c9ffa,1080453 5 1082452 13 1084451 21 1086449 27 108...,2000,1316,14,NaN
333101,ffc20018d3b87df69ae61f1c38a1257a,524898 7 526492 19 528086 27 529681 33 531275 ...,1600,1067,14,NaN
333155,ffd7077c30443a0d3f1cb1e0588bf41c,6156002 9 6161185 25 6166367 43 6171549 60 617...,5184,3456,14,NaN
333194,ffdd857f6f41f95adf6ae5c3ab8c3fc1,279920 6 280920 13 280951 19 281961 53 283003 ...,1044,700,14,NaN


,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
194,0024bec9284de7d329da7be884ec74bd,2171250 11 2174489 32 2177727 54 2180965 77 21...,3252,1860,21,NaN
195,0024bec9284de7d329da7be884ec74bd,3208361 4 3211612 10 3214863 16 3218114 23 322...,3252,1860,21,NaN
256,00375f11809c48821f56051b55b8e8dc,130080 4 130926 12 131773 20 132620 27 133466 ...,851,566,21,NaN
257,00375f11809c48821f56051b55b8e8dc,98615 6 99464 16 100314 20 101163 26 102012 31...,851,566,21,NaN
280,0039366c35ed742e5bfc6a278b699720,693894 21 695495 29 697095 32 698695 35 700296...,1600,1067,21,NaN
...,...,...,...,...,...,...
333151,ffd7077c30443a0d3f1cb1e0588bf41c,11247374 8 11252542 24 11257710 40 11262879 56...,5184,3456,21,NaN
333265,ffebc44ef1bbb5067632bfa26383e425,3062336 4 3065335 10 3068334 16 3071334 21 307...,3000,2000,21,NaN
333266,ffebc44ef1bbb5067632bfa26383e425,3443389 6 3446390 18 3449391 30 3452392 42 345...,3000,2000,21,NaN
333331,fff36676cdac2dd8d057783f0e9f24cc,730829 2 733081 6 735333 9 737585 13 739837 17...,2256,1500,21,NaN


,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
13,0002ec21ddb8477e98b2cbb87ea2e269,3077799 43 3080792 52 3083786 60 3086779 70 30...,3000,1997,23,NaN
14,0002ec21ddb8477e98b2cbb87ea2e269,2678617 8 2681610 18 2684607 25 2687605 31 269...,3000,1997,23,NaN
29,00048c3a2fb9c29340473c4cfc06424a,329326 3 330300 8 331276 12 332253 15 333231 1...,975,650,23,NaN
30,00048c3a2fb9c29340473c4cfc06424a,283509 4 284484 10 284549 2 285459 12 285520 7...,975,650,23,NaN
34,0006ea84499fd9a06fefbdf47a5eb4c0,236871 1 237870 3 238869 6 239869 8 240868 10 ...,1000,667,23,NaN
...,...,...,...,...,...,...
333371,fffb65463df408e308983ef743b77491,4079457 7 4082457 8 4085456 11 4088446 5 40884...,3000,2247,23,NaN
333379,fffc631acce2e28e1628de685d40c980,284625 4 285597 12 286569 20 287541 25 288513 ...,975,650,23,NaN
333380,fffc631acce2e28e1628de685d40c980,288552 15 289525 18 290475 2 290498 20 291447 ...,975,650,23,NaN
333390,fffe20b555b98c3c1f26c8dfff275cbc,3800472 4 3803465 12 3806458 20 3809451 29 381...,3000,2001,23,NaN


In [8]:
def create_mask_this_df(in_df):
    image_ids = in_df['ImageId'][:10]
    images_meta=[]

    for image_id in image_ids:
        img = mpimg.imread('./data/kaggle_FGVC/train/{}.jpg'.format(image_id))
        images_meta.append({
            'image': img,
            'shape': img.shape,
            'encoded_pixels': in_df[in_df['ImageId'] == image_id]['EncodedPixels'],
            'class_ids':  in_df[in_df['ImageId'] == image_id]['ClassId']
        })

    masks = []
    for image in images_meta:
        shape = image.get('shape')
        encoded_pixels = list(image.get('encoded_pixels'))
        class_ids = list(image.get('class_ids'))
        
        # Initialize numpy array with shape same as image size
        height, width = shape[:2]
        mask = np.zeros((height, width)).reshape(-1)
        
        # Iterate over encoded pixels and create mask
        for segment, (pixel_str, class_id) in enumerate(zip(encoded_pixels, class_ids)):
            splitted_pixels = list(map(int, pixel_str.split()))
            pixel_starts = splitted_pixels[::2]
            run_lengths = splitted_pixels[1::2]
            assert max(pixel_starts) < mask.shape[0]
            for pixel_start, run_length in zip(pixel_starts, run_lengths):
                pixel_start = int(pixel_start) - 1
                run_length = int(run_length)
                mask[pixel_start:pixel_start+run_length] = 255 - class_id * 4
        masks.append(mask.reshape((height, width), order='F'))  
    return masks, images_meta


mask_list = []


In [9]:
# 파일 분류
mask_list = []

my_category_list = [0,1,2,3,4,5,6,7,8,9,10,11,14,21,23]

my_category_dict = {0:'shirt, blouse',
                    1:'top, t-shirt, sweatshirt',
                    2:'sweater',
                    3:'cardigan',
                    4:'jacket',
                    5:'vest',
                    6:'pants',
                    7:'shorts',
                    8:'skirt',
                    9:'coat',
                    10:'dress',
                    11:'jumpsuit',
                    14:'hat',
                    21:'tights, stockings',
                    23:'shoe'}


for i in my_category_list:
    display(train_df[train_df['ClassId'] == i])
    print(' ')
  

my_sort_df = train_df[train_df['ClassId'] == 3]# 바꿔야할거
my_category_dict.get(3)

,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
1,00000663ed1ff0c4e0132b9b9ac53f6e,6323163 11 6328356 32 6333549 53 6338742 75 63...,5214,3676,0,"115,136,142,146,225,295,316,317"
119,0010ef2da53cd76ee26122a65f6e76ee,5153671 6 5158276 20 5162882 31 5167488 42 517...,4608,3072,0,"115,135,142,146,225,295,316,317"
222,002bc82e34205ecf66052749c8ff9e32,1303221 6 1305550 18 1307880 29 1310209 41 131...,2334,1522,0,"115,136,145,146,225,295,316,317"
239,0036f906cb15e240f113184b5eed8c69,116510 2 117157 6 117803 11 118445 20 119089 2...,650,433,0,"115,136,145,147,225,295,316,317"
283,0039366c35ed742e5bfc6a278b699720,752506 23 754090 70 755675 100 757265 110 7588...,1600,1067,0,"115,225,295,316,317"
...,...,...,...,...,...,...
333189,ffdc587a4b41cc4ff12c56e4fe06086b,249513 2 250713 6 251914 9 251929 5 253114 21 ...,1200,801,0,"147,316"
333211,ffe011e941a22c02ef0be02a922d44b7,6921499 3 6926776 9 6932054 15 6937331 19 6942...,5278,3519,0,"115,136,142,146,225,295,316,317"
333291,fff11e31ec0146bd469e5e3afa14a37a,289982 6 291001 16 292021 21 293043 25 294066 ...,1024,633,0,"115,136,295,316"
333309,fff132f7ce55db363a1afc4fda330ffc,2653567 3 2656560 11 2659556 15 2662553 18 266...,3000,1997,0,"115,136,142,146,225,295,316,317"


,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
10,0000fe7c9191fba733c8a69cfaf962b7,1343707 9 1346138 27 1348569 44 1351000 62 135...,2448,2448,1,"0,115,145,146,295,316,317"
24,00048c3a2fb9c29340473c4cfc06424a,257702 10 258670 26 259640 36 260572 6 260610 ...,975,650,1,"11,115,135,145,146,295,316,317"
31,0006ea84499fd9a06fefbdf47a5eb4c0,73327 6 74328 18 75330 37 76335 63 77340 88 78...,1000,667,1,"11,135,146,295"
37,000775b6b5e27b011dc8bba2d7b85211,309670 11 311259 33 312848 55 314440 72 316033...,1600,1067,1,"115,136,145,295,314,317"
54,000b3ec2c6eaffb491a5abb72c2e3e26,458445 17 460731 51 463017 85 465303 115 46759...,2310,1536,1,"0,115,136,145,147,295,314,317"
...,...,...,...,...,...,...
333273,ffec8295f37df6ea12eecbb60d2c23d4,2599516 1 2602970 3 2606425 4 2609879 6 261333...,3456,2304,1,"0,115,136,145,146,295,316,317"
333322,fff36676cdac2dd8d057783f0e9f24cc,1401684 4 1403933 12 1403962 4 1406137 4 14061...,2256,1500,1,"0,317"
333336,fff3ae4585d776922829ceb887789be4,122780 3 123500 6 123511 7 124220 22 124940 24...,720,480,1,"14,115,136,145,147,295,316,317"
333351,fff65379ba62e3f6498a20e9cc2a99fe,5913102 5 5917131 10 5921163 12 5925195 14 592...,4032,3024,1,"0,115,136,146,295,301,317"


,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
296,003a6ca3eaa7886da61ed2de2c5e8d27,5158311 19 5161042 23 5163772 28 5163869 5 516...,2733,3630,2,"136,147,295,316,317"
298,003ad8a37d2190bd944a8968fb0906e2,18556042 9 18561335 28 18566629 46 18571922 65...,5304,7952,2,"115,136,142,147,295,316,317"
314,003c831a9e84ee589d67c4d91786f525,48287 3 48882 9 49468 24 50049 44 50635 64 512...,600,371,2,"115,135,142,147,295,316,317"
580,00689c9d05028aad72aa2a9dfccbd491,8143995 5 8147985 15 8151974 26 8155964 36 815...,4000,6000,2,"115,136,295,316,317"
664,007e66e7c2864eb3c1ef95cd3ab52687,5093384 3 5098565 11 5103747 17 5108928 25 511...,5184,3456,2,"115,136,145,146,295,316,317,328"
...,...,...,...,...,...,...
332358,ff3c229ab653661ecd5c857ef20e77db,92626 4 93271 11 93917 14 94564 16 95211 18 95...,650,433,2,"115,136,142,149,295,316,317"
332663,ff73f17eae6e03385dac85aa2e0ad415,3324489 3 3327332 10 3330176 16 3333021 21 333...,2848,4288,2,"115,136,145,146,316"
332943,ffa9ee64af28f111006fbdabf6db09e2,3117 63 4848 153 6616 206 8384 259 10012 69 10...,1800,1183,2,"115,145,147,295,316,317,328"
333213,ffe011e941a22c02ef0be02a922d44b7,3285868 49 3291072 147 3296311 210 3301584 238...,5278,3519,2,"115,136,145,146,295,316,317"


,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
44,000aac3870ea7c59ca0333ffa5327323,8621972 2 8625970 5 8629968 9 8633966 13 86379...,4000,6000,3,"115,136,145,147,225,295,316,317"
478,00573a530f2e9e4c203681547e259f52,13173 8 13958 25 14744 41 15529 52 16314 57 17...,789,800,3,"115,136,145,146,295,316,317"
486,0058f13eef5b1a66e42666bbbb834ca9,81846 13 81929 39 82831 37 82893 76 83816 153 ...,1000,800,3,"115,145,146,225,281,312,319"
679,007fb0d71cfa24fe496363d5198f706d,36357 3 37004 7 37374 9 37650 13 38015 22 3823...,650,433,3,"115,137,142,149,295,316,317"
1192,00f43ac0d027955e5fe6c99bd412fd8f,3248208 5 3251198 16 3254187 27 3257177 37 326...,3000,2000,3,"115,136,145,149,225,295,316,317"
...,...,...,...,...,...,...
331727,feb527d59d79e3b191ea21a06a3a72b3,82532 1 83131 3 83729 5 84328 7 84926 10 85525...,600,450,3,"115,136,145,151,225,295,316,317"
332161,ff0c12c7e53961891d3930a0a61c698e,4203322 1 4206584 5 4209846 8 4213108 11 42163...,3264,4912,3,"147,295,316,317"
333148,ffd7077c30443a0d3f1cb1e0588bf41c,6492408 2 6497591 8 6502774 13 6507958 18 6513...,5184,3456,3,"115,149,295,316,328"
333269,ffebc44ef1bbb5067632bfa26383e425,1873710 1 1876708 3 1879707 4 1882705 6 188570...,3000,2000,3,"115,136,145,147,295,316,317"


,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
8,00000663ed1ff0c4e0132b9b9ac53f6e,4566382 8 4571592 25 4576803 41 4582013 58 458...,5214,3676,4,"17,115,136,145,149,225,295,311,317"
62,000c5bb8ca785bce8d00e3a6c7802a8c,322102 1 323125 2 324148 3 325171 4 326194 5 3...,1024,683,4,"115,135,142,146,308,317"
92,000e973c99dc090afd7898c93daf0dbc,4796962 5 4801882 14 4806801 24 4811721 33 481...,4928,3264,4,"17,115,136,142,146,295,316,328"
101,000e973c99dc090afd7898c93daf0dbc,2022977 2 2027903 8 2032828 14 2037754 19 2042...,4928,3264,4,"28,136,295,317"
167,001d7807a696231045860eaee97e825b,247017 4 248399 11 249780 16 251162 18 252543 ...,1383,900,4,"115,116,142,147,229,283,311,317"
...,...,...,...,...,...,...
333242,ffe8150a195e3aaa72640d7730a811a1,7279531 8 7283162 24 7286797 36 7290436 44 729...,3648,5472,4,"17,136,142,146,225,295,308,328"
333302,fff11e31ec0146bd469e5e3afa14a37a,195055 19 196068 56 197080 81 198092 94 199105...,1024,633,4,"17,115,136,142,148,225,295,311,317"
333319,fff132f7ce55db363a1afc4fda330ffc,2005308 93 2008300 194 2011293 206 2014288 217...,3000,1997,4,"17,115,136,142,147,225,295,311,322"
333334,fff3ae4585d776922829ceb887789be4,98199 15 98221 27 98905 69 99620 79 100334 90 ...,720,480,4,"21,115,136,142,146,225,295,298,317"


...

,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
1386,011ea8ad74020b797bb70dff32eb965d,1454723 7 1457712 22 1460700 37 1463689 51 146...,3000,2000,11,"115,125,141,155,295,301,325"
2846,02389d8de4ec5f040c639282732b53a1,162399 1 163478 3 164556 6 164589 8 165634 9 1...,1080,1080,11,"115,126,142,154,229,295,305,306,317"
3175,026a9d4e51588f0973eea33bb81c512a,1542006 4 1543999 12 1545991 20 1547983 28 154...,1995,3000,11,"115,295,317"
3776,02e0e61e9828870674129227de56e45c,1231735 9 1234134 26 1236533 44 1238932 61 124...,2408,1673,11,"115,125,141,154,295,316,317"
3854,02f6c36c2a99353edadbeb93931346f8,1294686 38 1297639 113 1300612 169 1303513 16 ...,3000,2000,11,"115,135,141,155,234,295,316,317"
...,...,...,...,...,...,...
330948,fe05e6d28b36fee9b3b9227d613e80b5,275926 9 277524 13 279120 19 280713 29 282306 ...,1600,1067,11,"115,125,137,141,155,229,295,298,317"
331168,fe31fb1a96780626756d473ba5a87af4,7803789 63 7809121 71 7814453 78 7819785 86 78...,5336,3837,11,"115,136,142,149,229,295,316,331"
331561,fe8702c3e0b29091a557a00eb6056570,2250178 7 2253666 21 2257154 35 2260642 49 226...,3500,2333,11,"115,125,142,155,229,295,316,317"
332808,ff89d127a396bc4e269161fcb0debafc,157993 7 158979 21 159966 34 160952 48 161938 ...,1000,667,11,"115,128,136,141,295,305,325"


,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
103,000e973c99dc090afd7898c93daf0dbc,5889525 12 5894431 36 5899346 51 5904270 57 59...,4928,3264,14,NaN
710,0084d5a59b959293507052f5679d1be5,6663362 49 6668544 54 6673726 59 6678908 65 66...,5184,3456,14,NaN
791,0091ae902cb738e4ca06038c2654fa00,714885 7 716680 20 718474 33 720269 46 722065 ...,1800,1200,14,NaN
1018,00cd5bb501e031aeeeeb337fb3f5b6d8,167360 2 168159 5 168959 5 169758 6 170557 7 1...,800,534,14,NaN
1088,00df4e94fea28af045f51846858a2e3e,7960563 6 7966169 18 7971775 31 7977381 43 798...,5616,3744,14,NaN
...,...,...,...,...,...,...
332930,ffa52a33c743c48512a85169251c9ffa,1080453 5 1082452 13 1084451 21 1086449 27 108...,2000,1316,14,NaN
333101,ffc20018d3b87df69ae61f1c38a1257a,524898 7 526492 19 528086 27 529681 33 531275 ...,1600,1067,14,NaN
333155,ffd7077c30443a0d3f1cb1e0588bf41c,6156002 9 6161185 25 6166367 43 6171549 60 617...,5184,3456,14,NaN
333194,ffdd857f6f41f95adf6ae5c3ab8c3fc1,279920 6 280920 13 280951 19 281961 53 283003 ...,1044,700,14,NaN


,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
194,0024bec9284de7d329da7be884ec74bd,2171250 11 2174489 32 2177727 54 2180965 77 21...,3252,1860,21,NaN
195,0024bec9284de7d329da7be884ec74bd,3208361 4 3211612 10 3214863 16 3218114 23 322...,3252,1860,21,NaN
256,00375f11809c48821f56051b55b8e8dc,130080 4 130926 12 131773 20 132620 27 133466 ...,851,566,21,NaN
257,00375f11809c48821f56051b55b8e8dc,98615 6 99464 16 100314 20 101163 26 102012 31...,851,566,21,NaN
280,0039366c35ed742e5bfc6a278b699720,693894 21 695495 29 697095 32 698695 35 700296...,1600,1067,21,NaN
...,...,...,...,...,...,...
333151,ffd7077c30443a0d3f1cb1e0588bf41c,11247374 8 11252542 24 11257710 40 11262879 56...,5184,3456,21,NaN
333265,ffebc44ef1bbb5067632bfa26383e425,3062336 4 3065335 10 3068334 16 3071334 21 307...,3000,2000,21,NaN
333266,ffebc44ef1bbb5067632bfa26383e425,3443389 6 3446390 18 3449391 30 3452392 42 345...,3000,2000,21,NaN
333331,fff36676cdac2dd8d057783f0e9f24cc,730829 2 733081 6 735333 9 737585 13 739837 17...,2256,1500,21,NaN


,ImageId,EncodedPixels,Height,Width,ClassId,AttributesIds
13,0002ec21ddb8477e98b2cbb87ea2e269,3077799 43 3080792 52 3083786 60 3086779 70 30...,3000,1997,23,NaN
14,0002ec21ddb8477e98b2cbb87ea2e269,2678617 8 2681610 18 2684607 25 2687605 31 269...,3000,1997,23,NaN
29,00048c3a2fb9c29340473c4cfc06424a,329326 3 330300 8 331276 12 332253 15 333231 1...,975,650,23,NaN
30,00048c3a2fb9c29340473c4cfc06424a,283509 4 284484 10 284549 2 285459 12 285520 7...,975,650,23,NaN
34,0006ea84499fd9a06fefbdf47a5eb4c0,236871 1 237870 3 238869 6 239869 8 240868 10 ...,1000,667,23,NaN
...,...,...,...,...,...,...
333371,fffb65463df408e308983ef743b77491,4079457 7 4082457 8 4085456 11 4088446 5 40884...,3000,2247,23,NaN
333379,fffc631acce2e28e1628de685d40c980,284625 4 285597 12 286569 20 287541 25 288513 ...,975,650,23,NaN
333380,fffc631acce2e28e1628de685d40c980,288552 15 289525 18 290475 2 290498 20 291447 ...,975,650,23,NaN
333390,fffe20b555b98c3c1f26c8dfff275cbc,3800472 4 3803465 12 3806458 20 3809451 29 381...,3000,2001,23,NaN


'cardigan'

In [10]:
# def make_folder(folder_name): # 폴더 생성 함수

#     if not os.path.isdir(folder_name):
#         os.mkdir(folder_name)

# for i in my_category_dict:
#     print(my_category_dict.get(i))
#     make_folder('./data/train_by_category_ver2/' + my_category_dict.get(i) + '/')

In [11]:
mask_list = []
# 메모리때문에 나눠서 해야함 ㅠㅠ
n = 0  # 바꿔야할거

for i in range(100):
    df_tmp = my_sort_df[i*10+n*10:i*10+10+n*10]
    # display(df_tmp)
    image_ids = df_tmp['ImageId'].unique()[:10]
    if image_ids == []:
        pass
    else:
        masks, images_meta = create_mask_this_df(df_tmp)
        
        mask_list = mask_list + masks
    print(n+i)
        
print(mask_list[0].shape)
print(len(mask_list))

/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
(4000, 6000)
1000


In [12]:
print(len(mask_list))

1000


In [13]:
k = 0
l = 0 # 바꿔야할거
for a in my_sort_df['ImageId'][l+0:l+1000]:
    # print(a)
    image = mpimg.imread('./data/kaggle_FGVC/train/{}.jpg'.format(a))
    try:
        if image.shape[0] > 6000 or image.shape[1] > 6000:
            pass
    #     print(image_copy.shape)
    #     print(my_mask.shape)
    #    th,my_mask = cv2.threshold(my_mask,0,255,cv2.THRESH_OTSU)
        else:
            image_copy = image.copy()
            my_mask = mask_list[k].copy()
            my_mask = (my_mask*1).astype('uint8')
            masked = cv2.bitwise_or(image_copy,image_copy,mask=my_mask)

            img = np.zeros(masked.shape, dtype="uint8")
            try:
                masked[np.where((masked==[0,0,0]).all(axis=2))] = [255,255,255]
            except:
                print('fail ', l)
        cv2.imwrite('./data/train_by_category_ver2/{}/{}.jpg'.format(my_category_dict.get(3), l), masked)  # 바꿔야할거
        
    except:
        print('image error : shape : ', image.shape)
    
    
    
    k += 1
    l += 1
    
    print(l)
    
  

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
fail  165
166


/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
fail  253
254
255
256
257
258
259
260
261
fail  261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411


In [14]:
print(k)
print(a)

1000
e332bcf8fe973b00755f8db6d4af64a7


In [15]:
image = mpimg.imread('./data/kaggle_FGVC/train/{}.jpg'.format(a))
image_copy = image.copy()
my_mask = mask_list[999].copy()
cv2.imwrite('./data/sample.jpg', image_copy)
cv2.imwrite('./data/sample_mask.jpg', my_mask)

True